In [1]:
import json

def save_json(data,name):
    with open(name+'.json', 'w') as f:
        json.dump(data, f)

def save_jsonl(data,name):
    with open(name+'.jsonl', 'w') as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write('\n')

def load_json(path):
    f = open (path, "r")
    data = json.loads(f.read())
    f.close()
    return data

def load_jsonl(path):
    f = open(path,"r").readlines()
    data=[]
    for i in f:
        d = json.loads(i)
        data.append(d)
    return data

In [2]:
# from ruamel_yaml import YAML
# import pytorch_lightning as pl
# from omegaconf import DictConfig
import IPython
import torch

if torch.cuda.is_available():

    device=torch.device("cuda:0")
    print("Running on the GPU")
    torch.cuda.empty_cache()
  #dataType=torch.float32
else:
    device=torch.device("cpu")
    print("Running on the CPU")
  #dataType=torch.float32

Running on the CPU


In [3]:
import pandas as pd
import sys
import glob
import pandas as pd
import os
if sys.platform == 'linux': path_sep = '/'
elif sys.platform[:3] == 'win': path_sep = '\\'
else: path_sep = '/'

DATA_PATH = os.path.join(os.getcwd(), 'data')
files_list = glob.glob(f'{DATA_PATH}/*.csv')


In [29]:
def get_columns(file_path: str):
    '''
    Function to read the first line of a csv file and return a list of column names.
    :param file_path: file path to csv file
    :return: column names
    '''
    with open(file_path, 'r') as file:
        contents = file.readline()
    return contents.strip().split(sep=',')

def get_product_rows(product_file_path):
    with open(product_file_path, 'r') as file:
        contents = file.readlines()
    columns = get_columns(product_file_path)
    for i, line in enumerate(contents[1:]):
        words = line.strip().split(sep=', ')
        if len(words) > len(columns):
           yield {columns[i]: words[i] if i == 0 else ' '.join(words[1:-len(columns)+2]) if i ==1 else words[-(len(columns) - i)] for i in range(len(columns))}
        else:
            yield {column: word for column, word in zip(columns, words)}



In [30]:
products = pd.DataFrame(get_product_rows("data/shwapno_dataset_1/products_all.csv"),)


In [31]:
# products = pd.DataFrame(get_product_rows("data/shwapno_dataset_1/products_all.csv"),)
columns = get_columns("data/shwapno_dataset_1/products_all.csv")
file_path = "data/shwapno_dataset_1/products_all.csv"
with open(file_path, 'r') as file:
    contents = file.readlines()
words = contents[1328].strip().split(sep=', ')

products.iloc[1327]

ProductCode                                       "3100877"
 ProductName      "X-Deshi Murgi Pack with Gila Koliza (B)"
 SubCategoryID                                       "3103"
 SubCategory                                           Meat
 CategoryID                                            "31"
 Category                                           Protein
 UnitPrice                                           0.0000
 VATID                                                   A1
 VAT                                                 0.0000
Name: 1327, dtype: object

In [21]:
row_dict

{'ProductCode': '"3100877"',
 ' ProductName': 'Koliza (B)"',
 ' VATID': 'A1',
 ' UnitPrice': '0.0000',
 ' Category': 'Protein',
 ' CategoryID': '"31"',
 ' SubCategory': 'Meat',
 ' SubCategoryID': '"3103"'}

In [24]:
# products.iloc[1327]
words

['"3100877"',
 '"X-Deshi Murgi Pack with Gila',
 'Koliza (B)"',
 '"3103"',
 'Meat',
 '"31"',
 'Protein',
 '0.0000',
 'A1',
 '0.0000']

## Data Preparation

In [4]:
# invoice_details = pd.read_csv("data/shwapno_dataset_1/invoice_202208_202307.csv")
customers = pd.read_csv("data/shwapno_dataset_1/customers_202208_202307.csv")
products = pd.read_csv("data/shwapno_dataset_1/products_all.csv")



# invoices_df.head()
#
# products.columns = products.iloc[0].to_list()
# products = products.drop([0])
# products.ProductCode = products.reset_index(drop=True).ProductCode.astype(str)
#
# def product_code2name(code):
#     return products[products.ProductCode==code]['ProductName'].tolist()[-1]
#
# def product_name2code(name: str):
#     return products[products.ProductName==name]['ProductCode'].tolist()[-1]
#
# def product_codes2names(codes: list):
#     print(codes)
#     return list(map(product_code2name, codes))
#
# def product_names2codes(names: list):
#     return list(map(product_name2code, names))

ParserError: Error tokenizing data. C error: Expected 9 fields in line 1328, saw 10


In [7]:
get_columns("data/shwapno_dataset_1/products_all.csv")



['ProductCode',
 ' ProductName',
 ' SubCategoryID',
 ' SubCategory',
 ' CategoryID',
 ' Category',
 ' UnitPrice',
 ' VATID',
 ' VAT']

In [147]:
def enquote_second_column(contents):
    # new_lines=[]
    for i, line in enumerate(contents):
        words = line.split(sep=', ')
        if len(words)==5:
            # x = line.split(sep='\", ')
            words[1] = '"' + words[1] +'"'
            # new_line = {'CustomerCode': words[0],
            #             'CustomerName': words[1],
            #             'DepotCode': words[2],
            #             'AvgNSI': words[], InvCnt}
            new_line = ','.join(words)
            yield new_line
        else:
            yield ','.join(words)
            # new_lines.append(new_line)
            # print(f"{i+1}th line: {line}")


In [182]:
def get_rows(file_path):
    with open(file_path, 'r') as file:
        contents = file.readlines()
    for i, line in enumerate(contents):
        words = line.split(sep=', ')
        if len(words)>5:
            yield {'CustomerCode': words[0], 'CustomerName': ' '.join(words[1:-3]), 'DepotCode': words[-3], 'AvgNSI': words[-2], 'InvCnt': words[-1]}
        else:
            yield {'CustomerCode': words[0], 'CustomerName': words[1], 'DepotCode': words[2], 'AvgNSI': words[3], 'InvCnt': words[4]}

In [197]:
## Read from customer csv file. and then convert the last column to integer, AvgNSI column to float, CustomerName customer code and DEPOTCOde to string

customers_raw = pd.DataFrame(get_rows("data/shwapno_dataset_1/customers_202208_202307.csv"),)

customers = pd.DataFrame(get_rows("data/shwapno_dataset_1/customers_202208_202307.csv"),)
customers.InvCnt = customers.InvCnt.apply(lambda x: int(x))
customers.AvgNSI = customers.AvgNSI.apply(lambda x: float(x))
customers.CustomerName = customers.CustomerName.apply(lambda x: x.strip('"'))
customers.CustomerCode = customers.CustomerCode.apply(lambda x: x.strip('"'))
customers.DepotCode = customers.DepotCode.apply(lambda x: x.strip('"'))


In [207]:
def get_columns(file_path):
    with open(file_path, 'r') as file:
        contents = file.readline()
    return contents.strip().split(sep=',')
get_columns('./data/shwapno_dataset_1/customers_updated_202208_202307.csv')

['CustomerCode', 'CustomerName', 'DepotCode', 'AvgNSI', 'InvCnt']

In [202]:
## Check whether all entries in the last column in customer dataframe are integers
#Check the size of the dataframe curently held in memory
# customers.info()
# customers.to_csv("data/shwapno_dataset_1/customers_updated_202208_202307.csv", index=False)
# customers_updated = pd.read_csv("data/shwapno_dataset_1/customers_updated_202208_202307.csv",)

# customers.to_csv("data/shwapno_dataset_1/customers_updated_202208_202307.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50861 entries, 0 to 50860
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerCode  50861 non-null  int64  
 1   CustomerName  50861 non-null  object 
 2   DepotCode     50861 non-null  object 
 3   AvgNSI        50861 non-null  float64
 4   InvCnt        50861 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.9+ MB


In [187]:
with open("data/shwapno_dataset_1/customers_202208_202307.csv", 'r') as file:
    contents = file.readlines()

for i, line in enumerate(contents):
    words = line.split(sep=', ')
    if len(words)>5:
        # x = line.split(sep='\", ')
       print(f"{i+1}th line: {customers.iloc[i][1]},\n{line}")

# x = contents[1312].split(sep=', ')
# ' '.join(x[1:-3]), x[-3]

1313th line: "Hasan Kabir Sipon(Journalist CANVAS )",
"01716382856", "Hasan Kabir Sipon(Journalist, CANVAS )", D055, 1319.563518, 3

1949th line: "shofipur kaliakoir,",
"01773611172", "shofipur, kaliakoir,", F090, 736.112051, 2

2320th line: "MD NANNA",
"01723718102", "MD, NANNA", F022, 551.793750, 1

2443th line: "Md Omar Farukh",
"01675545660", "Md, Omar Farukh", D063, 560.501521, 2

7371th line: "Dr. Md. Jahidur Rahman(Professor Department of Pa",
"01712525873", "Dr. Md. Jahidur Rahman(Professor, Department of Pa", F097, 1186.443602, 5

7673th line: "160/152 college",
"01511413734", "160/152, college", F113, 1331.940000, 1

8080th line: "164 lalbag",
"01937596065", "164, lalbag", D045, 802.261688, 2

9140th line: "Mrs Amina Nasrin",
"13130394", "Mrs, Amina Nasrin", D016, 1109.060136, 3

9277th line: "Mahmuda ",
"01729693485", "Mahmuda, ", F002, 787.442459, 2

10653th line: "Nazrul EPZ",
"01725927032", "Nazrul, EPZ", C011, 1556.667133, 5

12334th line: "House:04 Road:07,",
"017474041

In [148]:
with open("data/shwapno_dataset_1/temp2.csv", "w") as file:
    file.writelines(enquote_second_column(contents))

# new_contents.split(sep="\n")[1309*2:1314*2]

In [149]:
with open("data/shwapno_dataset_1/temp2.csv", "r") as file:
    contents_2 = file.readlines()

In [153]:
contents_2[561]

'"01938800900",""mr.jasim.khan h-30,r-02,b-b"",D049,470.860660,3\n'

In [151]:
customers = pd.read_csv("data/shwapno_dataset_1/temp2.csv",)

ParserError: Error tokenizing data. C error: Expected 5 fields in line 562, saw 7


In [51]:
print('\", ')

", 


In [7]:
invoice_details.ProductCode.unique()

array([2300011, 2300139, 2300438, ..., 3808633, 2303838, 2800614])

In [5]:
products

,ProductCode,ProductName,SubCategoryID,SubCategory,CategoryID,Category,UnitPrice,VATID,VAT
1,4214666,Sugar Loose Refined (Kg),2402,Essentials,24,Commodities,136.0000,A1,0.0000
2,2300035,S201372 SUPTA 3 COLOR SLIME,4203,Toys,42,Gift & Toys,99.0000,A6,4.9500
3,2603741,7 Up Pet Bottle 2000ml,2301,Carbonated Drink,23,Beverage & Tobacco,130.0000,A6,6.5000
4,2300782,Mr.White Detergent Powder 500g(B3G1F),2601,Fabric Care,26,Home Care,285.0000,A6,14.2500
5,2400643,Coca Cola 2.25 Ltr. pet,2301,Carbonated Drink,23,Beverage & Tobacco,140.0000,A6,7.0000
...,...,...,...,...,...,...,...,...,...
3833730,NaN,S200798 Baby Woolen Cap,3301,Clothes-33,33,Life Style,400.0,A7,30.0
3833788,NaN,Eon Food Ocean Ria Fish Sandwich 500g,2809,Snacks,28,Packaged Foods,690.0,A6,34.5
3837228,NaN,S201047 R_OVEN DISH- 8078-3,3801,Accessories,38,Home Accessories,775.0,A6,38.75
3840896,NaN,S202296 Combo Hijab 3 pcs Set,3301,Clothes-33,33,Life Style,449.0,A7,33.675


In [13]:
import numpy as np
return_condition = invoice_details.SalesQTY<=0
invoice_details = invoice_details[~return_condition]
invoice_details['SalesQTY'] = invoice_details['SalesQTY'].apply(lambda x: max((1, np.floor(x))))

invoice_details.ProductCode = invoice_details.ProductCode.astype(str)

invoice_details['new_product']

baskets = invoice_details.loc[invoice_details.index.repeat(invoice_details['SalesQTY'])].groupby('InvoiceNo')['ProductCode'].apply(list)


other_elements = invoice_details[['InvoiceNo','CustomerCode','PrepareDate', 'ProductNSI', 'NSI', 'SalesQTY', 'Cash_1_Card_0', 'InvoicePeriod', 'InvoiceDate','DepotCode']].drop_duplicates().set_index('InvoiceNo')
other_elements['PrepareDate'] = pd.to_datetime(other_elements.PrepareDate)
invoice_details.groupby(['CustomerCode', 'ProductCode']).InvoiceNo.count(sor    )

In [ ]:
from gensim.models import Word2Vec


In [ ]:
from torch.utils.data import Dataset


class ShwapnoDataset(Dataset):
    def __init__(self, csv_files, root_dir, transform=None):
        """
        Arguments:
            csv_files (dict): list of Path to the csv files of invoices, customers and products.
            root_dir (string): Directory with all the csv-s.
            transform (callable, optional): Optional transform to be applied on a sample.
        """

        self.invoices = pd.read_csv(csv_files['invoices'])
        self.customers = pd.read_csv(csv_files['customers'])
        self.products = pd.read_csv(csv_files['products'])

        return_condition = self.invoices.SalesQTY<=0
        self.invoices = self.invoices[~return_condition]
        self.invoices['SalesQTY'] = self.invoices['SalesQTY'].apply(lambda x: max((1, np.floor(x))))

        self.invoices.ProductCode = invoice_details.ProductCode.astype(str)
        self.baskets = self.invoices.loc[invoice_details.index.repeat(invoice_details['SalesQTY'])].groupby('InvoiceNo')['ProductCode'].apply(list)


        other_elements = invoice_details[['InvoiceNo','CustomerCode','PrepareDate', 'ProductNSI', 'NSI', 'SalesQTY', 'Cash_1_Card_0', 'InvoicePeriod', 'InvoiceDate','DepotCode']].drop_duplicates().set_index('InvoiceNo')
        other_elements['PrepareDate'] = pd.to_datetime(other_elements.PrepareDate)


        self.root_dir = root_dir
        self.transform = transform


    def __len__(self):
        return len(self.invoices)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def product_code2name(code):
        return products[products.ProductCode==code]['ProductName'].tolist()[-1]

    def product_name2code(name: str):
        return products[products.ProductName==name]['ProductCode'].tolist()[-1]

    def product_codes2names(codes: list):
        print(codes)
        return list(map(product_code2name, codes))

    def product_names2codes(names: list):
        return list(map(product_name2code, names))

    def create_basket(self):





class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))